# Evaluation Metrics

### Import Libraries and Dependencies

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

### Read Signal Data CSV and Import as Pandas DataFrame

In [2]:
# Read in data and set index
filepath = Path('../Resources/trading_signals.csv')
signals_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
signals_df = signals_df.set_index("date", drop=True)
signals_df.head()

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
9/22/14,101.06,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
9/23/14,102.64,NaN,NaN,0,0.0,0,0.0,0.0,100000.0,100000.0,NaN,NaN
9/24/14,101.75,NaN,NaN,0,0.0,0,0.0,0.0,100000.0,100000.0,0.0,0.0
9/25/14,97.87,NaN,NaN,0,0.0,0,0.0,0.0,100000.0,100000.0,0.0,0.0
9/26/14,100.75,NaN,NaN,0,0.0,0,0.0,0.0,100000.0,100000.0,0.0,0.0


In [13]:
signals_df.shape

(1258, 12)

### Prepare Portfolio Evaluation Metrics DataFrame

In [3]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [4]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,0.32975
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [5]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    (1 + signals_df['Portfolio Daily Returns'].mean())**252 - 1
)
portfolio_evaluation_df

,Backtest
Annual Return,0.068947
Cumulative Returns,0.32975
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [6]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    (1 + signals_df['Portfolio Daily Returns'].std())** 252 - 1
)
portfolio_evaluation_df

,Backtest
Annual Return,0.068947
Cumulative Returns,0.32975
Annual Volatility,7.82237
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [7]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annual Return,0.068947
Cumulative Returns,0.32975
Annual Volatility,7.82237
Sharpe Ratio,0.484084
Sortino Ratio,NaN


In [8]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

,Backtest
Annual Return,0.068947
Cumulative Returns,0.32975
Annual Volatility,7.82237
Sharpe Ratio,0.484084
Sortino Ratio,NaN


In [9]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean())
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean()
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annual Return,0.068947
Cumulative Returns,0.32975
Annual Volatility,7.82237
Sharpe Ratio,0.484084
Sortino Ratio,0.0432112


### Prepare Trade Evaluation Metrics DataFrame

In [10]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Calculate and Append Trade Evaluation Metrics

In [11]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2/12/15,8/3/15,500.0,126.46,118.44,63230.0,59220.0,-4010.0
1,AAPL,12/9/15,1/13/16,500.0,115.62,97.39,57810.0,48695.0,-9115.0
2,AAPL,4/25/16,6/14/16,500.0,105.08,97.46,52540.0,48730.0,-3810.0
3,AAPL,8/22/16,3/26/18,500.0,108.51,172.77,54255.0,86385.0,32130.0
4,AAPL,4/19/18,11/30/18,500.0,172.80,178.58,86400.0,89290.0,2890.0
5,AAPL,3/28/19,7/22/19,500.0,188.72,207.22,94360.0,103610.0,9250.0
